## Data Processing & Feature Engineering

This notebook script aims to carry out the following tasks:
- Take the clean data from the DB and aggregate it per station per hour.
- Calculate delays per measurement and classify the different delay values.
- Feature Engineering: calculate all the input predictors for the subsequent NN training.
- Data aggregation: Aggregate potential external data with correlation.
- Write all these processes in a new table in the DB as a ready-for-prediction table.

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from datetime import datetime
from itertools import groupby,accumulate
import statistics

In [2]:
#read clean data
#TEMPORARY FROM LOCAL CSV!!!!
#data = pd.read_csv("C:/Users/Usuari/Desktop/clean.csv", encoding = "UTF-8", error_bad_lines=False, sep=',',
#                  low_memory=False)

#data = pd.read_csv("C:/Users/gtorras/Desktop/clean.csv", encoding = "UTF-8", error_bad_lines=False, sep=',',
#                  low_memory=False)

In [3]:
#read data from DB!!
con = create_engine('postgresql://####:##########@#########:####/db')
data = pd.read_sql_table('clean_data', con)

In [4]:
data = data[['betriebstag', 'sendezeitpunkt', 'plan_einfahrtszeit', 'fahrtrichtung', 'haltestelle_nr' ]]
data.head()

,betriebstag,sendezeitpunkt,plan_einfahrtszeit,fahrtrichtung,haltestelle_nr
0,2021-04-30,07:22:57,07:21:00,-1,5064802
1,2021-04-30,07:23:57,07:22:00,-1,5002502
2,2021-03-31,06:49:35,06:50:00,-1,5622302
3,2021-03-31,06:51:05,06:51:00,-1,5003002
4,2021-03-31,06:52:55,06:53:00,-1,5045704


In [5]:
data.shape

(511234, 5)

##### Delay calculation
First, the delay in each measurement must be calculated. To do so, the measurement and planned time attributes (sendezeitpunkt & plan_einfahrtszeit) must be joined with the date (betriebstag) and converted to timestamp. From here, the difference of these two timestamps will be the resulting delay.

In [6]:
day = data['betriebstag'].tolist()
rec = data['sendezeitpunkt'].tolist()
plan = data['plan_einfahrtszeit'].tolist()

plan_timestamp = []
real_timestamp = []

for i in range(len(rec)):
    real_timestamp.append(day[i] + " " + rec[i])
    plan_timestamp.append(day[i] + " " + plan[i])

data['plan_timestamp'] = plan_timestamp
data['real_timestamp'] = real_timestamp


In [7]:
data['plan_time'] = pd.to_datetime(data['plan_timestamp'])
data['real_time'] = pd.to_datetime(data['real_timestamp'])

del data['plan_timestamp']
del data['real_timestamp']

data.head()

,betriebstag,sendezeitpunkt,plan_einfahrtszeit,fahrtrichtung,haltestelle_nr,plan_time,real_time
0,2021-04-30,07:22:57,07:21:00,-1,5064802,2021-04-30 07:21:00,2021-04-30 07:22:57
1,2021-04-30,07:23:57,07:22:00,-1,5002502,2021-04-30 07:22:00,2021-04-30 07:23:57
2,2021-03-31,06:49:35,06:50:00,-1,5622302,2021-03-31 06:50:00,2021-03-31 06:49:35
3,2021-03-31,06:51:05,06:51:00,-1,5003002,2021-03-31 06:51:00,2021-03-31 06:51:05
4,2021-03-31,06:52:55,06:53:00,-1,5045704,2021-03-31 06:53:00,2021-03-31 06:52:55


In [8]:
def CalcDelay():
    delay = []
    for i in range(len(data)):
        value = data['real_time'][i] - data['plan_time'][i]
        x = str(value) #timedelta to str format
        if x[1] == '1':
            delay.append(0) #no delay- bus gets there before
        else: 
            min_str = x[10:12]
            minutes = int(min_str)
            sec_str = x[13:15]
            seconds = int(sec_str)
            m = minutes * 60
            result = seconds + m
            delay.append(result)
            
    data['delay_sec'] = delay

In [9]:
#call function to calculate each delay
CalcDelay()

#drop sendezeitpunkt and plan_einfahrtszeit (timestamps are already calculated)
#betriebstag can be also deleted since it is already present in the timestamps
del data['sendezeitpunkt']
del data['plan_einfahrtszeit']
del data ['betriebstag']

In [10]:
data.head()

,fahrtrichtung,haltestelle_nr,plan_time,real_time,delay_sec
0,-1,5064802,2021-04-30 07:21:00,2021-04-30 07:22:57,117
1,-1,5002502,2021-04-30 07:22:00,2021-04-30 07:23:57,117
2,-1,5622302,2021-03-31 06:50:00,2021-03-31 06:49:35,0
3,-1,5003002,2021-03-31 06:51:00,2021-03-31 06:51:05,5
4,-1,5045704,2021-03-31 06:53:00,2021-03-31 06:52:55,0


##### Aggregatting data per station (Haltestelle nr)

Notice that the station number is used, since each station can have more than one direction. 

Now, the dataframe must be aggregated by station and hourly. 

In [11]:
#create day&hour field, so the dataset can be accordingly structured
day_hour = []

def Day_Hour(list):
    time = data['real_time'].tolist()

    for i in time:
        x = str(i)
        list.append(x[:13])

    data['day_hour'] = list
    
Day_Hour(day_hour)

In [12]:
#aggreagte the data
df = data.groupby(['day_hour', 'haltestelle_nr'], as_index=False).aggregate({'delay_sec' : 'mean'})
df

,day_hour,haltestelle_nr,delay_sec
0,2021-01-01 06,5000205,85.0
1,2021-01-01 06,5000301,36.0
2,2021-01-01 06,5000306,92.0
3,2021-01-01 06,5000403,113.0
4,2021-01-01 06,5000404,94.5
...,...,...,...
148479,2021-04-30 23,5091202,0.0
148480,2021-04-30 23,5097303,0.0
148481,2021-04-30 23,5653303,64.0
148482,2021-04-30 23,5653304,64.0


##### Delay Classification

For this study case, to make the predictions more accurate and understandable, the delay will be classified into 5 categories. With this, the neural network can learn better and the results are still useful. The different delay classes will be:
- 0 : On time
- 1: delay from 1-3 minutes
- 2: delay from 4-6 minutes
- 3: delay from 6-9 minutes
- 4: delay higher than 9 minutes

<b> Note: These delay figures compose the average delay per regional bus stop within each specific hour<b>

In [14]:
#delay classification 
delays = df['delay_sec'].tolist()
delay_class = []

def DelaysClass():
    for i in range(len(df)):
        if delays[i] == 0:
            delay_class.append(0)
        elif delays[i] > 0 and delays[i] <= 180:
            delay_class.append(1)
        elif delays[i] > 180 and delays[i] <= 360:
            delay_class.append(2)
        elif delays[i] > 360 and delays[i] <= 540:
            delay_class.append(3)
        else:
            delay_class.append(4)
            
    df['delay_class'] = delay_class
    
DelaysClass()

##### Feature Engineering

In this section, the predictors will be calculated and its correlation with the delay values will also be assessed.<br>
Predictor features:
- Previous hour aggregated delay per stop.
- Previous two hours aggregated delay per stop.
- Previous day at the same hour aggregated delay per stop.
- Previous two days at the same hour aggregated delay per stop.
- Previous week, same day and same hour aggregated delay per stop. 
- (maybe previous two weeks...)....

In [15]:
#lists definitions & extractions for the feature engineering
df_sorted = df.sort_values(by=['haltestelle_nr', 'day_hour']) #sort df according to stations

stat = df_sorted['haltestelle_nr'].tolist()
#delay = df_sorted['delay_class'].tolist()
delay = df_sorted['delay_sec'].tolist()
dh = df_sorted['day_hour'].tolist()

h = [x[11:13] for x in dh] #get hours within day_hour
hours = list(map(int, h)) #to int
d = [x[8:10] for x in dh] #get days
days = list(map(int, d)) 

Before starting with the calculation of predictor features, a creation of intermediate features must be done. What does this mean? Basically, an intermediate feature will be an attribute which will be used for the calculation of predictor features. For example, if it is the first day of the month, how will we calculate the previous day (e.g. how to go back from 1st to 31st, or 1st to 30th...)? Basically, in this case it will be a sequential number which will change each different day.

In [16]:
#first, create ID with date
year = [x[0:4] for x in dh] 
month = [x[5:7] for x in dh]

day_id = []

def DateID(lst):
    for i in range(len(df_sorted)):
        value_id = year[i] + month[i] + d[i]
        id_date = int(value_id)
        lst.append(id_date)

    df_sorted['date_id'] = lst
    
DateID(day_id)

In [17]:
#reorder the df accordingly
df_sorted = df_sorted.sort_values(by=['day_hour','date_id', 'haltestelle_nr'])
df_sorted

,day_hour,haltestelle_nr,delay_sec,delay_class,date_id
0,2021-01-01 06,5000205,85.0,1,20210101
1,2021-01-01 06,5000301,36.0,1,20210101
2,2021-01-01 06,5000306,92.0,1,20210101
3,2021-01-01 06,5000403,113.0,1,20210101
4,2021-01-01 06,5000404,94.5,1,20210101
...,...,...,...,...,...
148479,2021-04-30 23,5091202,0.0,0,20210430
148480,2021-04-30 23,5097303,0.0,0,20210430
148481,2021-04-30 23,5653303,64.0,1,20210430
148482,2021-04-30 23,5653304,64.0,1,20210430


In [18]:
#create sequential id for different dates
day_id = []
counter = 0 #to make the number sequential
date_id = df_sorted['date_id'].tolist()

def DayID(lst, column, count):
    for i in range(len(df_sorted)):
        if column[i] == column[i-1]:
            lst.append(count)
        elif column[i] != column[i-1]:
            count += 1
            lst.append(count)
        else:
            lst.append(np.nan)

    df_sorted['day_id'] = day_id

DayID(day_id, date_id, counter)

### Predictors Calculation

To calculate the delays I must resort de DF according to stations, so it can pick the previous values every time.

In [19]:
df_sorted = df_sorted.sort_values(by=['haltestelle_nr', 'day_hour']) #sort df according to stations to generate predictors

###### Feature to calculate: Previous hour delay value

In [20]:
#previous h delay calculation
prev_h_delay = []

def Prev_h_delay(list):
    for i in range(len(df_sorted)):
        if stat[i] == stat[i-1] and hours[i] == hours[i-1]+1 and day_id[i] == day_id[i-1]:
            list.append(delay[i-1])
        else:
            list.append(np.nan)

    df_sorted['prev_h_del'] = list

Prev_h_delay(prev_h_delay)

###### Feature to calculate: Previous 2 hours delay value

In [21]:
#previous 2h delay calculation
prev_2h_delay = []

def Prev_2h_delay(list):
    for i in range(len(df_sorted)):
        if stat[i] == stat[i-1] and hours[i] == hours[i-1]+2 and day_id[i] == day_id[i-1]:
                list.append(delay[i-1])
        elif stat[i] == stat[i-2] and hours[i] == hours[i-2]+2 and day_id[i] == day_id[i-2]:
                list.append(delay[i-2])
        else:
            list.append(np.nan)

    df_sorted['prev_2h_del'] = prev_2h_delay

Prev_2h_delay(prev_2h_delay)

In [22]:
#to check, lets check it with the sorted DF, since it will be clearer
#howerver, our data to work with is under df
df_sorted

,day_hour,haltestelle_nr,delay_sec,delay_class,date_id,day_id,prev_h_del,prev_2h_del
0,2021-01-01 06,5000205,85.000000,1,20210101,1,NaN,NaN
57,2021-01-01 07,5000205,75.000000,1,20210101,1,85.000000,NaN
102,2021-01-01 08,5000205,55.200000,1,20210101,1,75.000000,85.000000
164,2021-01-01 09,5000205,72.500000,1,20210101,1,55.200000,75.000000
203,2021-01-01 10,5000205,80.000000,1,20210101,1,72.500000,55.200000
...,...,...,...,...,...,...,...,...
148166,2021-04-30 18,5667802,41.000000,1,20210430,120,107.666667,27.000000
148240,2021-04-30 19,5667802,94.000000,1,20210430,120,41.000000,107.666667
148309,2021-04-30 20,5667802,59.666667,1,20210430,120,94.000000,41.000000
148368,2021-04-30 21,5667802,48.000000,1,20210430,120,59.666667,94.000000


At this point, it is time to go back over the days/weeks, and create the features regarding previous days delay values at the specific hour. 

In [23]:
#create hours id(to go back over the days and same hour)
df_sorted['hour_id'] = hours

In [24]:
#sorting hour_id and station
df_sorted = df_sorted.sort_values(by=['haltestelle_nr', 'hour_id', 'day_id']) #sorting to go back over the days (same hours)

###### Feature to calculate: Previous day same hour (delay value)

In [25]:
#previous day delay calculation
#columns are reordered so they have to be put to list again
hour = df_sorted['hour_id'].tolist()
stat = df_sorted['haltestelle_nr'].tolist()
day_id = df_sorted['day_id'].tolist()
#delay = df_sorted['delay_class'].tolist()
delay = df_sorted['delay_sec'].tolist()
prev_day_delay = []

def PrevDayDelay(lst):
    for i in range(len(df_sorted)):
        if stat[i] == stat[i-1] and hour[i] == hour[i-1] and day_id[i] == day_id[i-1]+1:
            lst.append(delay[i-1])  
        else:
            lst.append(np.nan)
    
    df_sorted['prev_day_del'] = lst
    
PrevDayDelay(prev_day_delay)

###### Feature to calculate: Previous 2 days same hour (delay value)

In [26]:
#calculate delay value same hour previous 2 days
prev_2day_delay = []

def Prev2DayDelay(lst):
    for i in range(len(df_sorted)):
        if stat[i] == stat[i-2] and hour[i] == hour[i-2] and day_id[i] == day_id[i-2]+2:
            lst.append(delay[i-2])  
        elif stat[i] == stat[i-1] and hour[i] == hour[i-1] and day_id[i] == day_id[i-1]+2:
            lst.append(delay[i-1])  
        else:
            lst.append(np.nan)
    
    df_sorted['prev_2day_del'] = lst
    
Prev2DayDelay(prev_2day_delay)

###### Features to calculate: Previous week, 2 weeks, 3 weeks, 4 weeks same hour (delay value)

In [27]:
#calculate delay value same hour previous week & same hour previous two weeks
prev_week_delay = []
prev_2week_delay = []
prev_3week_delay = []
prev_4week_delay = []

day1 = 1
day7 = 8
day14 = 15
day21 = 22
day28 = 29

def PrevWeekDelay(lst, days, range1, range2):
    for i in range(len(df_sorted)):
        x = 1
        for day in range(range1, range2): #for each week day (each row is one week day or more)
            if stat[i] == stat[i-day] and hour[i] == hour[i-day] and day_id[i] == day_id[i-day]+days:
                lst.append(delay[i-day])  
                x = 2 #change x value if there is delay value already filled
        if x == 1:
            lst.append(np.nan)
    
PrevWeekDelay(prev_week_delay, 7, day1, day7)
PrevWeekDelay(prev_2week_delay, 14, day1, day14)
PrevWeekDelay(prev_3week_delay, 21, day1, day21)
PrevWeekDelay(prev_4week_delay, 28, day1, day28)

#add to df
df_sorted['prev_week_del'] = prev_week_delay
df_sorted['prev_2week_del'] = prev_2week_delay
df_sorted['prev_3week_del'] = prev_3week_delay
df_sorted['prev_4week_del'] = prev_4week_delay

In [28]:
df_sorted.head(30)

,day_hour,haltestelle_nr,delay_sec,delay_class,date_id,day_id,prev_h_del,prev_2h_del,hour_id,prev_day_del,prev_2day_del,prev_week_del,prev_2week_del,prev_3week_del,prev_4week_del
5634,2021-01-06 00,5000205,64.0,1,20210106,6,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
9351,2021-01-09 00,5000205,98.0,1,20210109,9,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
14103,2021-01-13 00,5000205,34.0,1,20210113,13,NaN,NaN,0,NaN,NaN,64.0,NaN,NaN,NaN
16848,2021-01-15 00,5000205,75.0,1,20210115,15,NaN,NaN,0,NaN,34.0,NaN,NaN,NaN,NaN
18220,2021-01-16 00,5000205,62.0,1,20210116,16,NaN,NaN,0,75.0,NaN,98.0,NaN,NaN,NaN
22951,2021-01-20 00,5000205,62.0,1,20210120,20,NaN,NaN,0,NaN,NaN,34.0,64.0,NaN,NaN
24332,2021-01-21 00,5000205,107.0,1,20210121,21,NaN,NaN,0,62.0,NaN,NaN,NaN,NaN,NaN
25720,2021-01-22 00,5000205,93.0,1,20210122,22,NaN,NaN,0,107.0,62.0,75.0,NaN,NaN,NaN
27101,2021-01-23 00,5000205,73.0,1,20210123,23,NaN,NaN,0,93.0,107.0,62.0,98.0,NaN,NaN
31841,2021-01-27 00,5000205,36.0,1,20210127,27,NaN,NaN,0,NaN,NaN,62.0,34.0,64.0,NaN


###### Features to calculate: Mean / Median / Min and Max of all previous delay values per row

In [29]:
mean_del = []
median_del = []
min_delay = []
max_delay = []

def MeanMedianMaxMin():
    for i in range(len(df_sorted)):
        values = [prev_h_delay[i], prev_2h_delay[i], prev_day_delay[i], 
                            prev_2day_delay[i], prev_week_delay[i], prev_2week_delay[i]]
        
        mean = np.nanmean(values)
        mean_del.append(mean)

        median = np.nanmedian(values)
        median_del.append(median)

        min_del = np.nanmin(values)
        min_delay.append(min_del)

        max_del = np.nanmax(values)
        max_delay.append(max_del)
        
    df_sorted['mean_delay'] = mean_del
    df_sorted['median_delay'] = median_del
    df_sorted['min_delay'] = min_delay
    df_sorted['max_delay'] = max_delay

MeanMedianMaxMin()

<ipython-input-29-543b68f2d2d7>:11: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(values)
C:\Users\Usuari\anaconda3\envs\geo-env\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
<ipython-input-29-543b68f2d2d7>:17: RuntimeWarning: All-NaN axis encountered
  min_del = np.nanmin(values)
<ipython-input-29-543b68f2d2d7>:20: RuntimeWarning: All-NaN axis encountered
  max_del = np.nanmax(values)


In [30]:
df_sorted.head()

,day_hour,haltestelle_nr,delay_sec,delay_class,date_id,day_id,prev_h_del,prev_2h_del,hour_id,prev_day_del,prev_2day_del,prev_week_del,prev_2week_del,prev_3week_del,prev_4week_del,mean_delay,median_delay,min_delay,max_delay
5634,2021-01-06 00,5000205,64.0,1,20210106,6,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9351,2021-01-09 00,5000205,98.0,1,20210109,9,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,85.000000,85.00,85.0,85.0
14103,2021-01-13 00,5000205,34.0,1,20210113,13,NaN,NaN,0,NaN,NaN,64.0,NaN,NaN,NaN,74.666667,75.00,64.0,85.0
16848,2021-01-15 00,5000205,75.0,1,20210115,15,NaN,NaN,0,NaN,34.0,NaN,NaN,NaN,NaN,54.733333,55.20,34.0,75.0
18220,2021-01-16 00,5000205,62.0,1,20210116,16,NaN,NaN,0,75.0,NaN,98.0,NaN,NaN,NaN,75.175000,73.75,55.2,98.0


Having calculated all the features within this dataset, it is time to upload it as a table in the DB.

In [31]:
#write engineered dataframe into the DB
df_sorted.to_sql('data_eng', con, if_exists='replace')

#### Data Enrichment/Aggregation

In this subsection potential datasets which could potentially have an effect on the delay patterns are aggregated into the main dataframe. In this case, the precipitation data has been scrapped and restructured (see Data Aggregation notebook). 

As mentioned, scrapping and re-structuring is done in the Data Aggregation script. However, data join (enrichment) is done via SQL on the DBMS - see queries on the src/queries.sql doc-. Therefore, the enriched table with all the aggregated data is called <i> data_agg </i> within the DB.

#### Predictors evaluation and final selection

In this section, the correlation of the different created and aggregated features will be assessed. Consequently, the most relevant and useful ones will be kept for the subsequent prediction. 

In [32]:
df = pd.read_sql_table('data_agg', con)

In [33]:
df.corr()

,haltestelle_nr,date_id,day_id,hour_id,delay_sec,delay_class,prev_h_del,prev_2h_del,prev_day_del,prev_2day_del,prev_week_del,prev_2week_del,prev_3week_del,prev_4week_del,mean_delay,median_delay,min_delay,max_delay,prep_mm
haltestelle_nr,1.000000,-0.000769,-0.000962,0.003531,-0.016158,-0.002985,-0.020446,-0.024837,-0.018673,-0.017524,-0.015779,-0.014752,-0.015181,-0.015695,-0.032043,-0.035395,-0.027874,-0.016293,-0.001227
date_id,-0.000769,1.000000,0.984552,-0.000816,0.079316,0.065558,0.086680,0.095744,0.072974,0.068955,0.081525,0.077378,0.087780,0.067129,0.087260,0.113430,0.060705,0.041219,-0.008057
day_id,-0.000962,0.984552,1.000000,-0.000345,0.086772,0.071917,0.094380,0.102975,0.081874,0.077643,0.076337,0.073370,0.079141,0.076830,0.091165,0.115118,0.057785,0.047628,-0.005084
hour_id,0.003531,-0.000816,-0.000345,1.000000,-0.043836,-0.060722,-0.028543,0.004575,-0.044498,-0.046724,-0.038703,-0.039330,-0.037117,-0.026173,0.005255,-0.000348,-0.000338,0.006296,0.025238
delay_sec,-0.016158,0.079316,0.086772,-0.043836,1.000000,0.789455,0.449918,0.303937,0.269070,0.205767,0.218486,0.208188,0.194636,0.245518,0.310349,0.333625,0.248470,0.181745,0.057230
delay_class,-0.002985,0.065558,0.071917,-0.060722,0.789455,1.000000,0.346534,0.256081,0.246139,0.180815,0.207026,0.197038,0.192130,0.218566,0.282678,0.310294,0.238683,0.158626,0.041764
prev_h_del,-0.020446,0.086680,0.094380,-0.028543,0.449918,0.346534,1.000000,0.457799,0.199803,0.155580,0.156993,0.147545,0.142083,0.174400,0.240680,0.256946,0.194921,0.143010,0.046293
prev_2h_del,-0.024837,0.095744,0.102975,0.004575,0.303937,0.256081,0.457799,1.000000,0.168648,0.128993,0.139711,0.121058,0.118300,0.154557,0.212444,0.228575,0.177977,0.121878,0.043080
prev_day_del,-0.018673,0.072974,0.081874,-0.044498,0.269070,0.246139,0.199803,0.168648,1.000000,0.266747,0.175157,0.176017,0.164856,0.191775,0.573431,0.490528,0.324982,0.471351,-0.006032
prev_2day_del,-0.017524,0.068955,0.077643,-0.046724,0.205767,0.180815,0.155580,0.128993,0.266747,1.000000,0.142009,0.130412,0.129156,0.151007,0.547952,0.462605,0.312531,0.456154,-0.006145


The previous hour & 2h delay features have a defect. The neural network cannot be fed with NaN values. In this case, every day, in each first or secibd record of each station (first/second bus of the day), this value will be missing. Therefore, since it does not have a much superior correlation, it is convenient to also erase this feature. Once this is done, all the other features are compatible for the hourly prediction.

Also, it is observable that the aggregated feature regarding hourly precipitation does not correlate at all with the delay class patterns. According to this, unfortunately, it is not a useful feature for the classifier. Therefore, it will also be erased.

In [34]:
del df['prev_h_del']
del df['prev_2h_del']
del df['prep_mm']

In [35]:
df.corr()

,haltestelle_nr,date_id,day_id,hour_id,delay_sec,delay_class,prev_day_del,prev_2day_del,prev_week_del,prev_2week_del,prev_3week_del,prev_4week_del,mean_delay,median_delay,min_delay,max_delay
haltestelle_nr,1.000000,-0.000769,-0.000962,0.003531,-0.016158,-0.002985,-0.018673,-0.017524,-0.015779,-0.014752,-0.015181,-0.015695,-0.032043,-0.035395,-0.027874,-0.016293
date_id,-0.000769,1.000000,0.984552,-0.000816,0.079316,0.065558,0.072974,0.068955,0.081525,0.077378,0.087780,0.067129,0.087260,0.113430,0.060705,0.041219
day_id,-0.000962,0.984552,1.000000,-0.000345,0.086772,0.071917,0.081874,0.077643,0.076337,0.073370,0.079141,0.076830,0.091165,0.115118,0.057785,0.047628
hour_id,0.003531,-0.000816,-0.000345,1.000000,-0.043836,-0.060722,-0.044498,-0.046724,-0.038703,-0.039330,-0.037117,-0.026173,0.005255,-0.000348,-0.000338,0.006296
delay_sec,-0.016158,0.079316,0.086772,-0.043836,1.000000,0.789455,0.269070,0.205767,0.218486,0.208188,0.194636,0.245518,0.310349,0.333625,0.248470,0.181745
delay_class,-0.002985,0.065558,0.071917,-0.060722,0.789455,1.000000,0.246139,0.180815,0.207026,0.197038,0.192130,0.218566,0.282678,0.310294,0.238683,0.158626
prev_day_del,-0.018673,0.072974,0.081874,-0.044498,0.269070,0.246139,1.000000,0.266747,0.175157,0.176017,0.164856,0.191775,0.573431,0.490528,0.324982,0.471351
prev_2day_del,-0.017524,0.068955,0.077643,-0.046724,0.205767,0.180815,0.266747,1.000000,0.142009,0.130412,0.129156,0.151007,0.547952,0.462605,0.312531,0.456154
prev_week_del,-0.015779,0.081525,0.076337,-0.038703,0.218486,0.207026,0.175157,0.142009,1.000000,0.199414,0.196537,0.188348,0.531288,0.466309,0.329314,0.432876
prev_2week_del,-0.014752,0.077378,0.073370,-0.039330,0.208188,0.197038,0.176017,0.130412,0.199414,1.000000,0.194316,0.195449,0.537054,0.465370,0.328774,0.437599


After this, the dataframe can be filtered by keeping only the rows without null (nan) values, and re-written into a new DB table for the subsequent NN training/prediction.   --> TEMPORARY : I am writing a local csv instead for testing / development

Additionally, since the correlation of the features is not so relevant, a PCA will be run on the training and testing data in order to identify the best features and proceed accordingly (see neural network notebook). 

In [36]:
df = df.dropna()

In [37]:
#FOR DEVELOPMENT ONLY!#df.to_csv("C:/Users/Usuari/Desktop/ready_to_pred.csv")
#test.to_csv("C:/Users/gtorras/Desktop/ready_to_pred.csv")

In [38]:
#insert new table to dbdf.to_sql('ready_to_pred', con, if_exists='replace')